# 01-01 John Snow-Demo

## Hinweise zur Übung

Ziel dieser Demo ist eine Reproduktion der klassischen Analyse von Cholera-Daten durch John Snow in 1854. Der Datensatz ist frei verfügbar und wird hier schrittweise mit Python geladen, aufbereitet und visualisiert.

## Konfiguration des Notebooks

In [ ]:
# Ggf. fehlende Pakete installieren
!pip install --quiet pandas altair folium

In [ ]:
import os
import sys
import pandas as pd
import altair as alt
import folium

In [ ]:
# Konfiguration
base_url_quellen = "https://raw.githubusercontent.com/fau-lmi/lct-ehealth/main/08-Datenanalyse+Visualisierung/data"

## Rohdaten laden

Der vorliegende Datensatz enthält 3 Tabellen:
* `cholera_dates.csv`: Anzahl von Erkrankungen sowie Todesfällen pro Tag im Verlauf der Epidemie
  * date: Datum im Format YYYY-MM-DD
  * attacks: Anzahl der an diesem Datum neu an Cholera erkrankten
  * deaths: Anzahl der an diesem Datum an Cholera verstorbenen
* `cholera_deaths.csv`: Anzahl von Todesfällen zu den jeweiligen Geokoordinaten
  * fid: laufende Nummer der Adresse
  * deaths: Anzahl Cholera-Todesfälle an diesem Eintrag
  * lon: geographische Länge der Adresse
  * lat: geographische Breite der Adresse
* `water_pumps.csv': Orte von Wasserpumpen in London
  * fid: laufende Nummer der Adresse
  * lon: geographische Länge der Adresse
  * lat: geographische Breite der Adresse

Die Rohdaten liegen im CSV-Format vor und werden mit der `read_csv()`-Funktion des `pandas`-Packages in Dataframes geladen.


In [ ]:
cholera_dates  = pd.read_csv(base_url_quellen + "/JohnSnow/cholera_dates.csv",  index_col=0, parse_dates=['date'])
cholera_deaths = pd.read_csv(base_url_quellen + "/JohnSnow/cholera_deaths.csv", index_col=0)
water_pumps    = pd.read_csv(base_url_quellen + "/JohnSnow/water_pumps.csv",    index_col=0)

## Verlauf der Erkrankungen & Todesfälle plotten

Ein Python-Package für die flexible Visualisierung von Daten ist Altair/Vega-Lite. Es ist vergleichbar zum R-Package ggplot ("Grammar for Graphics"). Altair unterstützt eine Vielzahl unterschiedlicher Diagrammtypen, die bei Bedarf auch miteinander kombiniert werden können.

Im folgenden Chart sollen sowohl die Erkrankungen als auch die Todesfälle als Flächendiagramm (Area Charts) auf der Zeitachse dargestellt werden. Um beide Kennzahlen leicht auf einem gemeinsamen Diagramm darstellen zu können, werden die beiden Spalten `attacks` und `deaths` zunächst mit der `melt()`-Funktion aus einem "wide" in ein "long"-Format pivotiert.

In [ ]:
# Rohformat: Erkrankungen (attacks) und Todesfälle (deaths) liegen in separaten Spalten nebeneinander
cholera_dates.sort_values(by="date").head(5)

In [ ]:
# Pivotieren der Spalten in ein "long"-Format
cholera_dates_long = cholera_dates.melt(id_vars='date', value_vars=['attacks', 'deaths'], var_name='type', value_name='count')

# Ergebnis: die Attribute liegen jetzt in Zeilen untereinander
cholera_dates_long.sort_values(by=["date", "type"]).head(10)

Nachdem die Rohdaten pivotiert wurden, kann das Diagramm mit beiden Kennzahlen mit Altair generiert werden:

In [ ]:
# Flächendiagramm mit Altair erzeugen
chart = alt.Chart(cholera_dates_long).mark_area(opacity=0.5).encode(
    x='date:T',
    y=alt.Y('count:Q', stack=None),
    color=alt.Color('type:N', scale=alt.Scale(domain=['attacks', 'deaths'], range=['blue', 'red']))
).interactive()

# Diagramm ausgeben
chart.display()

## Geovisualisierung der Todesfälle

John Snow hatte die Adressen der Todesfälle gesammelt und händisch auf einer Karte von London eingezeichnet. In unserem Datensatz liegen die Adressen in geocodierter Form mit Längen- & Breitengraden vor. Mit dem Python-Package `folium` können mit minimalem Aufwand interaktive Landkarten erzeugt und darauf beliebige Marker eingezeichnet werden.

Die Karte muss zunächst mit einem gewünschten Mittelpunkt und Zoomfaktor (hier: Zentrum von London) angelegt werden. Anschließend wird über die Zeiten des Datensatzes iteriert und jeweils ein roter Kreis angelegt (Radius entspricht der Anzahl der dort registrierten Todesfälle). Mit dem letzten Befehl wird die Karte im Notebook ausgegeben.

In [ ]:
map = folium.Map(location=[51.513578, -0.136722], zoom_start=16)
for row in cholera_deaths.itertuples():
    folium.CircleMarker(location=[row.LAT,row.LON], radius=row.DEATHS*1.5, color="red", fill=True, Opacity=0.5).add_to(map)
map

## Erweiterung der Geovisualisierung um die Orte der Wasserpumpen

John Snow ging davon aus, dass die Cholera-Fälle auf verseuchte Wasserpumpen zurückzuführen sind. Er trug deshalb in die Karte der Todesfälle zusätzlich die im gleichen Stadtteil liegenden Wasserpumpen ein.

Wir nutzen hierzu die dritte CSV-Datei und ergänzen die Längen/Breitengrade der Pumpen als blaue Marker in der bestehenden Karte.

In [ ]:
for row in water_pumps.itertuples():
    folium.Marker(location=[row.LAT,row.LON]).add_to(map)
map

Es ist klar erkennbar, dass der Cluster der Todesfälle rund um die Pumpe in der Broad Street (heute: Broadwick Street) liegt. Diese Pumpe nahm John Snow persönlich außer Betrieb, indem er den Griff abmontierte. So konnte der Cholera-Ausbruch in diesem Stadtteil unterbrochen werden.